In [1]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))

In [2]:
A = np.pi*(0.16/2)**2
P0 = 100000
T0 = 610+273.15
R = 294
gamma = 1.34
critical_mass_flow_rate = ((A*P0*gamma**.5)/(R*T0)**.5)*(2/(gamma+1))**((gamma+1)/(2*gamma+2))
displayVariable('\\dot{m}_{cr}',critical_mass_flow_rate)
displayVariable('v_{max}',np.sqrt(((2*gamma)/(gamma-1))*R*T0))
displayVariable('a_0',np.sqrt(gamma*R*T0))
displayVariable('a_{cr}',np.sqrt(((2*gamma)/(gamma+1))*R*T0))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
m_dot,A,R,T0,P0,gamma,M = symbols('\\dot{m}_{cr} A R T_0 P_0 \\gamma M')
mach_function = (m_dot/A)*sp.sqrt(R*T0)/P0 - sp.sqrt(gamma)*M*(1+((gamma-1)/2)*M**2)**(-(gamma+1)/(2*(gamma-1)))
displayEquations(symbols('\\frac{\\partial{f}}{\\partial{M}}'),mach_function.diff(M).simplify())
mach_function = mach_function.subs({gamma:1.34,P0:100000,T0:883.15,A:np.pi*(0.19/2)**2,R:294,m_dot:critical_mass_flow_rate})
displayEquations(symbols('f(M)'),mach_function)
mach_function_prime = mach_function.diff(M)
displayEquations(symbols('\\frac{\\partial{f}}{\\partial{M}}'),mach_function_prime)
displayVariable('\\delta_f',(mach_function/mach_function_prime).simplify())
mach_function = lambdify(M,mach_function)
mach_function_prime = lambdify(M,mach_function_prime)
step_fuction = lambda M: mach_function(M)/mach_function_prime(M)
intial_mach_guess = np.linspace(0.01,.99,1000)
mach_newton_rapson = {'M_i':[],'f_i':[],'step':[],'M_min':[]}
iterations = 0
for mach in intial_mach_guess:
  mach_number = mach
  while 1e-8 < mach_function(mach_number) or mach_function(mach_number) < -1e-8:
    mach_newton_rapson['M_i'].append(mach_number)
    mach_newton_rapson['f_i'].append(mach_function(mach_number))
    mach_newton_rapson['step'].append(step_fuction(mach_number))
    mach_number = mach_number - step_fuction(mach_number)
    iterations += 1
    if abs(mach_number) >=20:
      # print('mach number is a unstable minimum')
      break
    if iterations > 1e4:
      # print('guess did not converge')
      break
  min_value = 1
  min_index = 0
  for index,value in enumerate(mach_newton_rapson['f_i']):
    if value < min_value:
      min_value = value
      min_index = index
  closest_approximation = mach_newton_rapson['M_i'][min_index]
  mach_newton_rapson['M_min'].append(closest_approximation)
linear_interpolation = (max(mach_newton_rapson['M_min'])-min(mach_newton_rapson['M_min']))*0.5 + min(mach_newton_rapson['M_min'])
displayVariable('M',linear_interpolation)
# display(pd.DataFrame.from_dict(mach_newton_rapson))
# displayVariable('M',closest_approximation)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
# Problem 4
M1 = 2.4
gamma = 1.33
degree_to_rad = np.pi/180
M1n = M1*np.sin(28*degree_to_rad)
w = M1*np.cos(28*degree_to_rad)
M2n = (M1n**2 + 2/(gamma-1))/(((2*gamma)/(gamma-1))*M1n**2 - 1)
displayVariable('M_{2\,n}',M2n)
M2 = (M2n**2+w**2)**.5
displayVariable('M_2',M2)
a = 10/np.tan(28*degree_to_rad)
displayVariable('a',a)
l = 10-2*a*np.tan(8*degree_to_rad)
displayVariable('l',l)
xs = a + l/np.tan(32*degree_to_rad)
ys = xs*np.tan(10*degree_to_rad)
displayEquations(Matrix([symbols('x_s'),symbols('y_s')]),Matrix([xs,ys]))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>